# Capstone Mini-Project: Data Wrangling

### Daniel Lee
### June 29, 2018

For the data wrangling portion, I divided my work into the following steps. For the first nine steps, I am reading in and cleaning the nine different datasets from 2016 county level data found from the web. In the tenth step, I combine the nine datasets together. In the eleventh step, I obtain mortality data caused by opioid overdose from 1999 - 2016 (not county level) with categories gender, ethnicity, and age-group.

1. prescription_rates: opioid prescription rates
2. education
3. population
4. poverty
5. unemployment
6. death rates from drug overdose
7. overall death rates
* county level race data
* county level age data
8. merge seven datasets together
9. Mortality rates from opioid overdose 1999 - 2016 (gender, ethnicity, age-group information included)

In [2]:
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup

## 1. Opioid Prescription Rates

In [2]:
# scrape prescription data from internet using BeautifulSoup
url = 'https://www.cdc.gov/drugoverdose/maps/rxcounty2016.html'
r = requests.get(url)
r.status_code
soup = BeautifulSoup(r.content, "html.parser")
stat_table = soup.find_all('table', class_ = 'table table-striped')
stat_table = stat_table[0]

# write the table into a txt with context manager
with open ('prescribing_stats.txt', 'w') as r:
    for row in stat_table.find_all('tr'): # find all rows in the table
        for cell in row.find_all('td'):
            r.write(cell.text.ljust(30))
        r.write('\n')

In [3]:
# read in the prescription data into a pandas df
prescription_rates = pd.read_csv('prescribing_stats.txt', delimiter = r"[ \t]{2,}", header = None, engine = 'python', na_values = '–')
prescription_rates.columns = ['county', 'state', 'county_code', 'opioid_prescription_rate']

In [4]:
prescription_rates.head()

,county,state,county_code,opioid_prescription_rate
0,"Aleutians East, AK",AK,2013,NaN
1,"Aleutians West, AK",AK,2016,NaN
2,"Anchorage, AK",AK,2020,66.3
3,"Bethel, AK",AK,2050,NaN
4,"Bristol Bay, AK",AK,2060,NaN


In [5]:
prescription_rates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3143 entries, 0 to 3142
Data columns (total 4 columns):
county                      3143 non-null object
state                       3143 non-null object
county_code                 3143 non-null int64
opioid_prescription_rate    2962 non-null float64
dtypes: float64(1), int64(1), object(2)
memory usage: 98.3+ KB


In [6]:
# remove null prescription_rates values from prescription_rates (2962/3143 non-null values)
prescription_rates_no_na = prescription_rates.loc[prescription_rates.opioid_prescription_rate.notnull(), :]

In [7]:
prescription_rates_no_na.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2962 entries, 2 to 3142
Data columns (total 4 columns):
county                      2962 non-null object
state                       2962 non-null object
county_code                 2962 non-null int64
opioid_prescription_rate    2962 non-null float64
dtypes: float64(1), int64(1), object(2)
memory usage: 115.7+ KB


## 2. Education

In [8]:
# download data using bash command curl
!curl -O -J https://www.ers.usda.gov/webdocs/DataFiles/48747/Education.xls?v=43125

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1529k  100 1529k    0     0   461k      0  0:00:03  0:00:03 --:--:--  461k


In [9]:
# change name of downloaded file using bash command mv
!mv Education.xls?v=43125 education.xls

In [10]:
# load education data from Excel file
xl = pd.ExcelFile('education.xls')
xl.sheet_names

['Education 1970 to 2016']

In [11]:
# load a sheet into a pandas df by index
education = xl.parse(0, skiprows = 4)

# select columns that only contain relevant data for 2016
education = education[list(education.columns[:3]) + [column for column in education.columns if '2016' in column]]

In [12]:
education.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3283 entries, 0 to 3282
Data columns (total 11 columns):
FIPS Code                                                                     3283 non-null int64
State                                                                         3283 non-null object
Area name                                                                     3283 non-null object
Less than a high school diploma, 2012-2016                                    3273 non-null float64
High school diploma only, 2012-2016                                           3273 non-null float64
Some college or associate's degree, 2012-2016                                 3273 non-null float64
Bachelor's degree or higher, 2012-2016                                        3273 non-null float64
Percent of adults with less than a high school diploma, 2012-2016             3273 non-null float64
Percent of adults with a high school diploma only, 2012-2016                  3273 non-null float

In [13]:
# drop all na values (3273/3283 non null values)
education_no_na = education.dropna(how = 'any')

In [14]:
education_no_na.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3273 entries, 0 to 3282
Data columns (total 11 columns):
FIPS Code                                                                     3273 non-null int64
State                                                                         3273 non-null object
Area name                                                                     3273 non-null object
Less than a high school diploma, 2012-2016                                    3273 non-null float64
High school diploma only, 2012-2016                                           3273 non-null float64
Some college or associate's degree, 2012-2016                                 3273 non-null float64
Bachelor's degree or higher, 2012-2016                                        3273 non-null float64
Percent of adults with less than a high school diploma, 2012-2016             3273 non-null float64
Percent of adults with a high school diploma only, 2012-2016                  3273 non-null float

## 3. Population

In [15]:
# download data using bash command curl
!curl -O -J https://www.ers.usda.gov/webdocs/DataFiles/48747/PopulationEstimates.xls?v=43209

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 4382k  100 4382k    0     0   677k      0  0:00:06  0:00:06 --:--:--  758k


In [16]:
# rename the downloaded file to education.xls using bash command mv
!mv PopulationEstimates.xls?v=43209 population.xls

In [17]:
# load population information from Excel file
xl = pd.ExcelFile('population.xls')
xl.sheet_names

['Population Estimates 2010-17', 'Variable Descriptions']

In [18]:
# load a sheet into a pandas df by index 
population = xl.parse(0, skiprows = 2)

# select columns that only contain relevant data for 2016
population = population[list(population.columns[:3]) + [column for column in population.columns if '2016' in column]]

In [19]:
population.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3273 entries, 0 to 3272
Data columns (total 19 columns):
FIPS                        3273 non-null int64
State                       3273 non-null object
Area_Name                   3273 non-null object
POP_ESTIMATE_2016           3273 non-null int64
N_POP_CHG_2016              3194 non-null float64
Births_2016                 3194 non-null float64
Deaths_2016                 3194 non-null float64
NATURAL_INC_2016            3194 non-null float64
INTERNATIONAL_MIG_2016      3194 non-null float64
DOMESTIC_MIG_2016           3194 non-null float64
NET_MIG_2016                3194 non-null float64
RESIDUAL_2016               3194 non-null float64
GQ_ESTIMATES_2016           3193 non-null float64
R_birth_2016                3194 non-null float64
R_death_2016                3194 non-null float64
R_NATURAL_INC_2016          3194 non-null float64
R_INTERNATIONAL_MIG_2016    3194 non-null float64
R_DOMESTIC_MIG_2016         3194 non-null float64

In [20]:
# drop all na values (3273/3283 non null values)
pop_2016_no_na = population.dropna(how = 'any')

In [21]:
pop_2016_no_na.head()

,FIPS,State,Area_Name,POP_ESTIMATE_2016,N_POP_CHG_2016,Births_2016,Deaths_2016,NATURAL_INC_2016,INTERNATIONAL_MIG_2016,DOMESTIC_MIG_2016,NET_MIG_2016,RESIDUAL_2016,GQ_ESTIMATES_2016,R_birth_2016,R_death_2016,R_NATURAL_INC_2016,R_INTERNATIONAL_MIG_2016,R_DOMESTIC_MIG_2016,R_NET_MIG_2016
1,1000,AL,Alabama,4860545,9687.0,59095.0,52077.0,7018.0,4678.0,-1964.0,2714.0,-45.0,118989.0,12.170229,10.724918,1.445311,0.963404,-0.404473,0.558931
2,1001,AL,Autauga County,55278,440.0,666.0,505.0,161.0,15.0,266.0,281.0,-2.0,455.0,12.096335,9.172146,2.924189,0.272440,4.831269,5.103709
3,1003,AL,Baldwin County,207509,4646.0,2312.0,2045.0,267.0,169.0,4205.0,4374.0,5.0,2275.0,11.267825,9.966567,1.301258,0.823643,20.493601,21.317244
4,1005,AL,Barbour County,25774,-490.0,272.0,269.0,3.0,-5.0,-488.0,-493.0,0.0,2977.0,10.453899,10.338599,0.115300,-0.192167,-18.755525,-18.947692
5,1007,AL,Bibb County,22633,72.0,290.0,207.0,83.0,20.0,-32.0,-12.0,1.0,2201.0,12.833562,9.160508,3.673054,0.885073,-1.416117,-0.531044


## 4. Poverty Estimates

In [22]:
# download data using bash command curl
!curl -O -J https://www.ers.usda.gov/webdocs/DataFiles/48747/PovertyEstimates.xls?v=43125

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  876k  100  876k    0     0   342k      0  0:00:02  0:00:02 --:--:--  342k


In [23]:
# rename the downloaded file to education.xls
!mv PovertyEstimates.xls?v=43125 poverty.xls

In [24]:
# load poverty estimate data from Excel file
xl = pd.ExcelFile('poverty.xls')
xl.sheet_names

['Poverty Data 2016', 'Variable Descriptions']

In [25]:
# load a sheet into a pandas df by index 
poverty = xl.parse(0, skiprows = 3)

In [26]:
# select columns that only contain relevant data for 2016
poverty = poverty[list(poverty.columns[:3]) + [column for column in poverty.columns if '2016' in column]]

In [27]:
poverty.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3194 entries, 0 to 3193
Data columns (total 30 columns):
FIPStxt            3194 non-null int64
State              3194 non-null object
Area_Name          3194 non-null object
POVALL_2016        3193 non-null float64
CI90LBAll_2016     3193 non-null float64
CI90UBALL_2016     3193 non-null float64
PCTPOVALL_2016     3193 non-null float64
CI90LBALLP_2016    3193 non-null float64
CI90UBALLP_2016    3193 non-null float64
POV017_2016        3193 non-null float64
CI90LB017_2016     3193 non-null float64
CI90UB017_2016     3193 non-null float64
PCTPOV017_2016     3193 non-null float64
CI90LB017P_2016    3193 non-null float64
CI90UB017P_2016    3193 non-null float64
POV517_2016        3193 non-null float64
CI90LB517_2016     3193 non-null float64
CI90UB517_2016     3193 non-null float64
PCTPOV517_2016     3193 non-null float64
CI90LB517P_2016    3193 non-null float64
CI90UB517P_2016    3193 non-null float64
MEDHHINC_2016      3193 non-null flo

In [28]:
# remove columns with only 52 entries and drop rows with na values (3193/3194 non null values)
poverty_no_na = poverty[list(poverty.columns[:-7])].dropna(how = 'any')

In [29]:
poverty_no_na.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3193 entries, 0 to 3193
Data columns (total 23 columns):
FIPStxt            3193 non-null int64
State              3193 non-null object
Area_Name          3193 non-null object
POVALL_2016        3193 non-null float64
CI90LBAll_2016     3193 non-null float64
CI90UBALL_2016     3193 non-null float64
PCTPOVALL_2016     3193 non-null float64
CI90LBALLP_2016    3193 non-null float64
CI90UBALLP_2016    3193 non-null float64
POV017_2016        3193 non-null float64
CI90LB017_2016     3193 non-null float64
CI90UB017_2016     3193 non-null float64
PCTPOV017_2016     3193 non-null float64
CI90LB017P_2016    3193 non-null float64
CI90UB017P_2016    3193 non-null float64
POV517_2016        3193 non-null float64
CI90LB517_2016     3193 non-null float64
CI90UB517_2016     3193 non-null float64
PCTPOV517_2016     3193 non-null float64
CI90LB517P_2016    3193 non-null float64
CI90UB517P_2016    3193 non-null float64
MEDHHINC_2016      3193 non-null flo

## 5. Unemployment Rate

In [30]:
# download data using bash command curl
!curl -O -J https://www.ers.usda.gov/webdocs/DataFiles/48747/Unemployment.xls?v=43244

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1456k  100 1456k    0     0   472k      0  0:00:03  0:00:03 --:--:--  472k


In [31]:
# rename the downloaded file to education.xls
!mv Unemployment.xls?v=43244 unemployment.xls

In [32]:
# load unemployment data from Excel file
xl = pd.ExcelFile('unemployment.xls')
xl.sheet_names

['Unemployment Med HH Inc', 'Variable Descriptions']

In [33]:
# load a sheet into a pandas df by index 
unemployment = xl.parse(0, skiprows = 7)

In [34]:
# select columns that only contain relevant data for 2016
unemployment = unemployment[list(unemployment.columns[:3]) + [column for column in unemployment.columns if '2016' in column]]

In [35]:
unemployment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3275 entries, 0 to 3274
Data columns (total 9 columns):
FIPStxt                                      3275 non-null int64
State                                        3275 non-null object
Area_name                                    3275 non-null object
Civilian_labor_force_2016                    3272 non-null float64
Employed_2016                                3272 non-null float64
Unemployed_2016                              3272 non-null float64
Unemployment_rate_2016                       3272 non-null float64
Median_Household_Income_2016                 3193 non-null float64
Med_HH_Income_Percent_of_State_Total_2016    3192 non-null float64
dtypes: float64(6), int64(1), object(2)
memory usage: 230.4+ KB


In [36]:
# remove all null values from median_household_income_2016 column (3193/3275 nonnull values)
unemployment_no_na = unemployment.loc[unemployment.Median_Household_Income_2016.notnull(), :]

In [37]:
unemployment_no_na.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3193 entries, 0 to 3195
Data columns (total 9 columns):
FIPStxt                                      3193 non-null int64
State                                        3193 non-null object
Area_name                                    3193 non-null object
Civilian_labor_force_2016                    3193 non-null float64
Employed_2016                                3193 non-null float64
Unemployed_2016                              3193 non-null float64
Unemployment_rate_2016                       3193 non-null float64
Median_Household_Income_2016                 3193 non-null float64
Med_HH_Income_Percent_of_State_Total_2016    3192 non-null float64
dtypes: float64(6), int64(1), object(2)
memory usage: 249.5+ KB


In [38]:
unemployment_no_na.head()

,FIPStxt,State,Area_name,Civilian_labor_force_2016,Employed_2016,Unemployed_2016,Unemployment_rate_2016,Median_Household_Income_2016,Med_HH_Income_Percent_of_State_Total_2016
0,0,US,United States,158921892.0,151183680.0,7738212.0,4.869192,57617.0,NaN
1,1000,AL,Alabama,2173175.0,2045624.0,127551.0,5.900000,46309.0,100.0
2,1001,AL,"Autauga County, AL",25918.0,24593.0,1325.0,5.100000,54487.0,117.7
3,1003,AL,"Baldwin County, AL",90500.0,85656.0,4844.0,5.400000,56460.0,121.9
4,1005,AL,"Barbour County, AL",8402.0,7700.0,702.0,8.400000,32884.0,71.0


# 6. Opioid Overdose Death Rate

Drug overdose death data can be downloaded from https://wonder.cdc.gov/controller/saved/D77/D39F042. From here, I agree to the CDC terms and generate a dataset with the preset search query information that is already preloaded. Then, I export the data file and save it into my working directory. I also added this file to my github account https://raw.githubusercontent.com/danielhanbitlee/Springboard/master/capstone_project/data/2016.txt so that it's readily accessible for anyone to access. Note that statistics representing zero to nine death counts in a given county are suppressed at the region, county, and state level. These counties with suppressed statistics are assumed to have zero mortality rate, since the number of deaths tend to be negligible when compared to the county population size.

In [40]:
# download data using bash command curl
!curl -O https://raw.githubusercontent.com/danielhanbitlee/Springboard/master/capstone_project/data/2016.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  176k  100  176k    0     0   124k      0  0:00:01  0:00:01 --:--:--  124k


In [41]:
# remove the comments and notes from the txt file and select only the data. Then save the file into drug_overdose.txt
!sed -n 2,3148p '2016.txt' > drug_overdose.txt

In [42]:
# load drug overdose death data into pandas df
death_rates = pd.read_csv('drug_overdose.txt', 
                             delimiter = r"[\t]", 
                             engine = 'python', 
                             header = None, 
                             na_values=['Missing'])

# insert column names into df
death_rates.columns = ['county','fips_code', 'deaths', 'population', 'crude_opioid_mortality_rate']

# remove double quotes from data
death_rates.county = death_rates.county.str.replace('"', "")
death_rates.fips_code = death_rates.fips_code.str.replace('"', "")

# replace 'Suppressed' with 0 for all columns
death_rates = death_rates.replace('Suppressed', 0)

# convert deaths column to numeric
death_rates.deaths = pd.to_numeric(death_rates.deaths)

# manually calculate crude_opioid_mortality_rate to fill in unreliable values with given data
death_rates['crude_opioid_mortality_rate'] = round(death_rates.deaths / death_rates.population * 100000, 1)

# fill na values of crude_opioid_mortality_rate column with -1
death_rates.crude_opioid_mortality_rate = death_rates.crude_opioid_mortality_rate.fillna(-1)

# convert appropriate columns into numeric data type
death_rates.fips_code = pd.to_numeric(death_rates.fips_code)
death_rates.population = pd.to_numeric(death_rates.population)
death_rates.crude_opioid_mortality_rate = pd.to_numeric(death_rates.crude_opioid_mortality_rate)

In [43]:
# check for any rows with missing values
death_rates[death_rates.isnull().any(axis=1)]

,county,fips_code,deaths,population,crude_opioid_mortality_rate
88,"Prince of Wales-Outer Ketchikan Census Area, AK",2201,NaN,NaN,-1.0
91,"Skagway-Hoonah-Angoon Census Area, AK",2232,NaN,NaN,-1.0
96,"Wrangell-Petersburg Census Area, AK",2280,NaN,NaN,-1.0
2919,"Bedford city, VA",51515,NaN,NaN,-1.0
2924,"Clifton Forge city, VA",51560,NaN,NaN,-1.0


In [44]:
death_rates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3147 entries, 0 to 3146
Data columns (total 5 columns):
county                         3147 non-null object
fips_code                      3147 non-null int64
deaths                         3142 non-null float64
population                     3142 non-null float64
crude_opioid_mortality_rate    3147 non-null float64
dtypes: float64(3), int64(1), object(1)
memory usage: 123.0+ KB


In [45]:
death_rates.head()

,county,fips_code,deaths,population,crude_opioid_mortality_rate
0,"Autauga County, AL",1001,0.0,55416.0,0.0
1,"Baldwin County, AL",1003,18.0,208563.0,8.6
2,"Barbour County, AL",1005,0.0,25965.0,0.0
3,"Bibb County, AL",1007,0.0,22643.0,0.0
4,"Blount County, AL",1009,0.0,57704.0,0.0


In [46]:
# save dataframe as csv for plotting purposes
death_rates.to_csv('death_rates.csv', index = False)

## 7. Overall Death Rate (All Causes)

Overall death rate for 2016 for all causes of death by county data can be downloaded from https://wonder.cdc.gov/controller/saved/D77/D41F026. From here, I agree to the CDC terms and generate a dataset with the preset search query information that is already preloaded. Then, I export the data file and save it into my working directory. I also added this file to my github account https://raw.githubusercontent.com/danielhanbitlee/Springboard/master/capstone_project/data/overall_death_rate_2016.txt so that it's readily accessible for anyone to access.

In [47]:
# read in the data for all deaths
overall_death_rate = pd.read_csv("https://raw.githubusercontent.com/danielhanbitlee/Springboard/master/capstone_project/data/overall_death_rate_2016.txt",
            delimiter = r"[\t]", 
            engine = 'python', 
            header = None, 
            na_values=['Missing'])[1:]

overall_death_rate = overall_death_rate.iloc[:3147, 0:-1] # get rid of rows that contain notes; column that is all NaN

# rename columns
overall_death_rate.columns = ['county', 'fips_code', 'deaths', 'population', 'crude_mortality_rate_all_causes']

# reset index to start from 0 and also drop the index column afterwards
overall_death_rate.reset_index(inplace = True, drop = True)

# remove double quotes from data
overall_death_rate.county = overall_death_rate.county.str.replace('"', "")
overall_death_rate.fips_code = overall_death_rate.fips_code.str.replace('"', "")

# replace 'Suppressed' with 0 for all columns
overall_death_rate = overall_death_rate.replace('Suppressed', 0)

# convert deaths column to numeric
overall_death_rate.deaths = pd.to_numeric(overall_death_rate.deaths)
overall_death_rate.population = pd.to_numeric(overall_death_rate.population)

# manually calculate crude_mortality_rate_all_causes to fill in unreliable values with given data
overall_death_rate['crude_mortality_rate_all_causes'] = round(overall_death_rate.deaths / overall_death_rate.population * 100000,
                                                              1)

# fill na values of crude_mortality_rate_all_causes column with -1
overall_death_rate.crude_mortality_rate_all_causes = overall_death_rate.crude_mortality_rate_all_causes.fillna(-1)

# convert appropriate columns into numeric data type
overall_death_rate.fips_code = pd.to_numeric(overall_death_rate.fips_code)
overall_death_rate.population = pd.to_numeric(overall_death_rate.population)
overall_death_rate.crude_mortality_rate_all_causes = pd.to_numeric(overall_death_rate.crude_mortality_rate_all_causes)

In [48]:
overall_death_rate.head()

,county,fips_code,deaths,population,crude_mortality_rate_all_causes
0,"Autauga County, AL",1001,520.0,55416.0,938.4
1,"Baldwin County, AL",1003,1974.0,208563.0,946.5
2,"Barbour County, AL",1005,256.0,25965.0,985.9
3,"Bibb County, AL",1007,239.0,22643.0,1055.5
4,"Blount County, AL",1009,697.0,57704.0,1207.9


In [49]:
# check for any rows with missing values
overall_death_rate[overall_death_rate.isnull().any(axis=1)]

,county,fips_code,deaths,population,crude_mortality_rate_all_causes
88,"Prince of Wales-Outer Ketchikan Census Area, AK",2201,NaN,NaN,-1.0
91,"Skagway-Hoonah-Angoon Census Area, AK",2232,NaN,NaN,-1.0
96,"Wrangell-Petersburg Census Area, AK",2280,NaN,NaN,-1.0
2919,"Bedford city, VA",51515,NaN,NaN,-1.0
2924,"Clifton Forge city, VA",51560,NaN,NaN,-1.0


In [50]:
overall_death_rate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3147 entries, 0 to 3146
Data columns (total 5 columns):
county                             3147 non-null object
fips_code                          3147 non-null int64
deaths                             3142 non-null float64
population                         3142 non-null float64
crude_mortality_rate_all_causes    3147 non-null float64
dtypes: float64(3), int64(1), object(1)
memory usage: 123.0+ KB


## 8. County Level Race Data 2016

County race data for 2016 can be downloaded from https://wonder.cdc.gov/controller/saved/D134/D41F352. From here, I agree to the CDC terms and generate a dataset with the preset search query information that is already preloaded. Then, I export the data file and save it into my working directory. I also added this file to my github account https://raw.githubusercontent.com/danielhanbitlee/Springboard/master/capstone_project/data/county_race_2016.txt so that it's readily accessible for anyone to access.

In [267]:
# read in data, remove rows with notes at the bottom of the document
county_race = pd.read_csv('https://raw.githubusercontent.com/danielhanbitlee/Springboard/master/capstone_project/data/county_race_2016.txt',
                          delimiter = r"[\t]", 
                          engine = 'python').drop(columns = ['Population', '"Race"']).iloc[:12588, :]

# rename the columns
county_race.columns = ['county', 'fips', 'race', 'population']

# remove missing population data (these counties are no longer counties)
county_race = county_race[county_race.population != 'Missing'] 

# replace double quotes in dataframe
for column in county_race.columns:
    county_race['{}'.format(column)] = county_race['{}'.format(column)].str.replace('"', "")

# convert fips and population columns to numeric
for column in ['fips', 'population']:
    county_race['{}'.format(column)] = pd.to_numeric(county_race['{}'.format(column)])

# pivot data so that columns are race
county_race_pivoted = county_race.pivot(index = 'fips', 
                                              columns='race', 
                                              values='population').sort_values(by = 'fips')

# calculate the total population for each county
total_population = county_race_pivoted.sum(axis = 1)

# calculate percentage of race by total population of county
for column in county_race_pivoted.columns:
    county_race_pivoted['{}_pct'.format(column)] = county_race_pivoted['{}'.format(column)] / \
                                                   total_population * 100

# drop population count columns and only keep the percentages
county_race_pct = county_race_pivoted.drop(columns = county_race_pivoted.columns[:4])

county_race_pct.head()

race,American Indian or Alaska Native_pct,Asian or Pacific Islander_pct,Black or African American_pct,White_pct
fips,,,,
1001,0.498051,1.396709,19.833622,78.271618
1003,0.862090,1.256695,9.563537,88.317679
1005,0.673984,0.647025,48.503755,50.175236
1007,0.379808,0.353310,22.130460,77.136422
1009,0.672397,0.441910,1.885485,97.000208


## 9. County Level Age Data 2016

County age data for 2016 can be downloaded from https://wonder.cdc.gov/controller/saved/D134/D41F353. From here, I agree to the CDC terms and generate a dataset with the preset search query information that is already preloaded. Then, I export the data file and save it into my working directory. I also added this file to my github account https://raw.githubusercontent.com/danielhanbitlee/Springboard/master/capstone_project/data/county_age_2016.txt so that it's readily accessible for anyone to access.

In [10]:
# read in data, remove rows with notes at the bottom of the document
county_age = pd.read_csv('https://raw.githubusercontent.com/danielhanbitlee/Springboard/master/capstone_project/data/county_age_2016.txt',
                          delimiter = r"[\t]", 
                          engine = 'python').drop(columns = ['Population', '"Age Group"'])

# rename the columns
county_age.columns = ['county', 'fips', 'age', 'population']

county_age = county_age.dropna(how = 'any') # drop notes at the bottom of the document

# remove missing population data (these counties are no longer counties)
county_age = county_age[county_age.population != 'Missing'] 

# replace double quotes in dataframe
for column in county_age.columns:
    county_age['{}'.format(column)] = county_age['{}'.format(column)].str.replace('"', "")
    county_age['{}'.format(column)] = county_age['{}'.format(column)].str.replace('-', "_")
    county_age['{}'.format(column)] = county_age['{}'.format(column)].str.replace(' ', "_")

# convert fips and population columns to numeric
for column in ['fips', 'population']:
    county_age['{}'.format(column)] = pd.to_numeric(county_age['{}'.format(column)])    

# pivot data so that columns are age
county_age_pivoted = county_age.pivot(index = 'fips', 
                                              columns='age', 
                                              values='population').sort_values(by = 'fips')

# calculate the total population for each county
total_population = county_age_pivoted.sum(axis = 1)

# calculate percentage of age by total population of county
for column in county_age_pivoted.columns:
    county_age_pivoted['{}_pct'.format(column)] = county_age_pivoted['{}'.format(column)] / \
                                                   total_population * 100

# drop population count columns and only keep the percentages
county_age_pct = county_age_pivoted.drop(columns = [column for column in county_age_pivoted.columns if 'pct' not in column])

# replace double underline in column names for county_age_pct df
county_age_pct.columns = [column.replace('__', '_') if'__' in column else column for column in county_age_pct.columns]

county_age_pct.head()

,10_14_years_pct,15_19_years_pct,1_4_years_pct,20_24_years_pct,25_29_years_pct,30_34_years_pct,35_39_years_pct,40_44_years_pct,45_49_years_pct,50_54_years_pct,55_59_years_pct,5_9_years_pct,60_64_years_pct,65_69_years_pct,70_74_years_pct,75_79_years_pct,80_84_years_pct,85+_years_pct,<_1_year_pct
fips,,,,,,,,,,,,,,,,,,,
1001,7.297531,7.012415,4.626823,5.944132,6.429551,6.249098,6.440378,6.871662,7.118883,7.165800,6.804894,6.620832,5.557962,4.805471,3.847264,2.869207,1.894760,1.311895,1.131442
1003,6.317036,5.977091,4.493127,5.324530,5.746465,5.662558,6.010654,6.078739,6.599445,6.927403,7.201661,5.957912,6.897676,6.769178,5.161510,3.553363,2.250639,1.973504,1.097510
1005,5.981128,5.619103,4.251878,6.408627,7.136530,7.090314,6.227614,6.173695,6.385519,6.870788,6.339303,6.204506,6.339303,6.192952,4.698633,3.158097,2.183709,1.783170,0.955132
1007,5.723623,5.803118,4.469372,6.103432,7.763989,7.194276,6.712891,6.810052,7.340017,7.529921,6.518571,5.546968,5.895862,5.383562,3.833414,2.976637,1.815131,1.422073,1.157090
1009,6.607861,6.417233,4.663455,5.599265,5.935464,5.758700,6.138223,6.301123,6.947525,6.937127,6.711840,6.476154,6.372175,6.094898,4.737973,3.292666,2.181824,1.679260,1.147234


## 10. Merge All the Datasets Together

In [289]:
# merge all the data sets together and drop redundant columns

# merge opioid death rate and overall death rate
death_rates_combined = pd.merge(death_rates, overall_death_rate, on = ['county', 'fips_code', 'population']).\
                                                                        drop(['deaths_x', 'deaths_y'], axis = 1)

# merge above to prescription rates
death_prescribe = pd.merge(death_rates_combined, prescription_rates_no_na, left_on = 'fips_code', right_on = 'county_code').\
                                                                                    drop(['county_x', 'county_code'], axis = 1)

# merge above to poverty data
dpp = pd.merge(death_prescribe, poverty_no_na, left_on = 'fips_code', right_on = 'FIPStxt').drop(['FIPStxt', 'Area_Name'],\
                                                                                                                 axis = 1)

# merge above to population estimates data
dppp = pd.merge(dpp, pop_2016_no_na, left_on = 'fips_code', right_on = 'FIPS').drop(['State_x', 'FIPS', 'State_y', 'Area_Name'],\
                                                                                                                        axis = 1)

# merge above to education data
dpppe = pd.merge(dppp, education_no_na, left_on = 'fips_code', right_on = 'FIPS Code').drop(['FIPS Code', 'State', 'Area name'],\
                                                                                                                        axis = 1)

# merge above to unemployment data
dpppe_unemployment = pd.merge(dpppe, unemployment_no_na, left_on = 'fips_code', right_on = 'FIPStxt').\
                                                            drop(['FIPStxt', 'State', 'Area_name'], axis = 1)

# calculate mortality rate excluding deaths caused by opioid - mortality_rate_excluding_opioid
dpppe_unemployment['mortality_rate_excluding_opioid'] = dpppe_unemployment.crude_mortality_rate_all_causes -\
                                                 dpppe_unemployment.crude_opioid_mortality_rate

# merge race pct data
dpppe_unemployment_race = pd.merge(dpppe_unemployment, county_race_pct, left_on = 'fips_code', right_on = 'fips')

# merge age pct data
merged_data = pd.merge(dpppe_unemployment_race, county_age_pct, left_on = 'fips_code', right_on = 'fips')

# remove the redundant state information from the county_y column and create a new column county
merged_data['county'] = merged_data.county_y.str.partition(',')[0]

# remove columns representing confidence intervals
merged_data = merged_data.loc[:, [column for column in merged_data.columns if 'CI' not in column]]

# rename column names
merged_data.columns = pd.Series(merged_data.columns).str.lower()\
                                                    .str.replace(' ', '_')\
                                                    .str.replace(',','')\
                                                    .str.replace("'s", "")\
                                                    .str.replace('percent', '%')\
                                                    .str.replace('pct', '%')\
                                                    .str.replace('_of_adults', '')\
                                                    .str.replace('completing_', '')\
                                                    .str.replace('_with_', '_')\
                                                    .str.replace('_a_', '_')\
                                                    .str.replace('n_pop_chg_2016', 'pop_chg_2016')\
                                                    .str.replace('%p', '%_p')\
                                                    .str.replace('<', 'less_than')
merged_data.columns = pd.Series(merged_data.columns).apply(lambda x: re.sub(r"^r_", "rate_", x)).tolist()

# remove poverty columns with overlapping information
merged_data = merged_data.loc[:, [column for column in merged_data.columns if 'pov' not in column or column == '%_povall_2016']]

# remove columns with education total count data (as opposed to percentage data) as well as other redundant education columns
merged_data = merged_data.drop(columns=['bachelor_degree_or_higher_2012-2016',
                                        'high_school_diploma_only_2012-2016',
                                        'less_than_high_school_diploma_2012-2016',
                                        'some_college_or_associate_degree_2012-2016',
                                        '%_bachelor_degree_or_higher_2012-2016',
                                        '%_less_than_high_school_diploma_2012-2016',
                                        '%_some_college_or_associate_degree_2012-2016'])

# remove other unnecessary columns
merged_data = merged_data.drop(columns=['unemployed_2016',
                                        'births_2016', 
                                        'deaths_2016', 
                                        'domestic_mig_2016',
                                        'international_mig_2016',
                                        'med_hh_income_%_of_state_total_2016',
                                        'natural_inc_2016',
                                        'net_mig_2016',
                                        'residual_2016',
                                        'rate_natural_inc_2016',
                                        'rate_net_mig_2016',
                                        'civilian_labor_force_2016',
                                        'employed_2016',
                                        'medhhinc_2016',
                                        'pop_estimate_2016',
                                        'county_y',
                                        'rate_death_2016'])

# drop crude_mortality_rate_all_causes column
merged_data = merged_data.drop(labels = 'crude_mortality_rate_all_causes', axis = 1)

In [290]:
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2962 entries, 0 to 2961
Data columns (total 39 columns):
fips_code                               2962 non-null int64
population                              2962 non-null float64
crude_opioid_mortality_rate             2962 non-null float64
state                                   2962 non-null object
opioid_prescription_rate                2962 non-null float64
%_povall_2016                           2962 non-null float64
pop_chg_2016                            2962 non-null float64
gq_estimates_2016                       2962 non-null float64
rate_birth_2016                         2962 non-null float64
rate_international_mig_2016             2962 non-null float64
rate_domestic_mig_2016                  2962 non-null float64
%_high_school_diploma_only_2012-2016    2962 non-null float64
unemployment_rate_2016                  2962 non-null float64
median_household_income_2016            2962 non-null float64
mortality_rate_excluding_opi

In [291]:
merged_data.head()

,fips_code,population,crude_opioid_mortality_rate,state,opioid_prescription_rate,%_povall_2016,pop_chg_2016,gq_estimates_2016,rate_birth_2016,rate_international_mig_2016,...,55_59_years_%,5_9_years_%,60_64_years__%,65_69_years_%,70_74_years_%,75_79_years_%,80_84_years_%,85+_years_%,less_than_1_year_%,county
0,1001,55416.0,0.0,AL,129.6,14.0,440.0,455.0,12.096335,0.272440,...,6.804894,6.620832,5.557962,4.805471,3.847264,2.869207,1.894760,1.311895,1.131442,Autauga
1,1003,208563.0,8.6,AL,123.8,12.0,4646.0,2275.0,11.267825,0.823643,...,7.201661,5.957912,6.897676,6.769178,5.161510,3.553363,2.250639,1.973504,1.097510,Baldwin
2,1005,25965.0,0.0,AL,92.7,30.0,-490.0,2977.0,10.453899,-0.192167,...,6.339303,6.204506,6.339303,6.192952,4.698633,3.158097,2.183709,1.783170,0.955132,Barbour
3,1007,22643.0,0.0,AL,97.2,20.0,72.0,2201.0,12.833562,0.885073,...,6.518571,5.546968,5.895862,5.383562,3.833414,2.976637,1.815131,1.422073,1.157090,Bibb
4,1009,57704.0,0.0,AL,56.9,14.0,-28.0,489.0,11.723635,1.076838,...,6.711840,6.476154,6.372175,6.094898,4.737973,3.292666,2.181824,1.679260,1.147234,Blount


In [292]:
# write a csv file of the merged data
merged_data.to_csv('merged_data.csv', index = False)

## 11. Mortality Rates from Opioid Overdose 1999 - 2016

Here, I obtain dataset containing mortality rates and death numbers due to opioid overdose from 1999 to 2016. This data is not divided by counties. However, it includes columns that contain age-group, ethnicity, and gender. This data will be used primarily for visualization purposes. The data is obtained from the CDC website. The saved query can be found in this link: https://wonder.cdc.gov/controller/saved/D77/D39F124. I also posted this dataset on github for easy access: https://raw.githubusercontent.com/danielhanbitlee/Springboard/master/capstone_project/data/Opioid%20Overdose%20Deaths.txt 

In [55]:
# read data in as csv
opioid_deaths = pd.read_csv(\
    'https://raw.githubusercontent.com/danielhanbitlee/Springboard/master/capstone_project/data/Opioid%20Overdose%20Deaths.txt',\
                            delimiter = '\t')

# relabel year-group categories
opioid_deaths['ten_year_groups_relabel'] = opioid_deaths['Ten-Year Age Groups Code'].\
                    apply(lambda x: (type(x) == str and len(str(x)) < 5 and x != 'NS' and x != '85+' and '0' + str(x)) or x)

# remove confidence interval columns
opioid_deaths = opioid_deaths.drop(list(opioid_deaths.columns[opioid_deaths.columns.str.contains('Confidence')]), axis = 1)

# remove notes column
opioid_deaths = opioid_deaths.drop('Notes', axis = 1)

# drop null values in this data
opioid_deaths = opioid_deaths.dropna()

# convert year column to integer
opioid_deaths.Year = opioid_deaths['Year'].map(lambda x: int(x))

opioid_deaths.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1728 entries, 0 to 1880
Data columns (total 12 columns):
Ten-Year Age Groups         1728 non-null object
Ten-Year Age Groups Code    1728 non-null object
Race                        1728 non-null object
Race Code                   1728 non-null object
Gender                      1728 non-null object
Gender Code                 1728 non-null object
Year                        1728 non-null int64
Year Code                   1728 non-null float64
Deaths                      1728 non-null float64
Population                  1728 non-null object
Crude Rate                  1728 non-null object
ten_year_groups_relabel     1728 non-null object
dtypes: float64(2), int64(1), object(9)
memory usage: 175.5+ KB


In [56]:
# save the dataframe as opioid_deaths.csv
opioid_deaths.to_csv('opioid_deaths.csv', index = None)